In [ ]:
import copy

from gensim.models import Word2Vec, FastText, KeyedVectors, fasttext, word2vec

import os

#os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from torch import nn
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, DataCollatorWithPadding,DistilBertForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput

from transformers import DataCollatorForTokenClassification
import torch
from torch.autograd import Variable
from pathlib import Path
import re
from datasets import load_dataset

from sklearn.model_selection import train_test_split
import copy
import time
# import wandb

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, classification_report
from umap import UMAP

In [ ]:
df = pd.read_csv('DataML3.csv',encoding='utf-8')
df.columns

In [ ]:
data=df[df['OverallSatisfactionScore']<=0.5]
data.head(5)

In [ ]:
classes_score=np.sort(data['OverallSatisfactionScore'].drop_duplicates().values)
classes_score_dict={}
for i,cls in enumerate(classes_score):
    classes_score_dict[str(cls)]=i
print(len(classes_score_dict))
data['OverallSatisfactionScore']=data['OverallSatisfactionScore'].map(lambda x: classes_score_dict[str(x)])
data.head(5)

In [ ]:
classes_service_type=np.sort(data['QHC_FUP_SERVICE_TYPE'].drop_duplicates().values)
classes_service_type_dict={}
for i,cls in enumerate(classes_service_type):
    classes_service_type_dict[str(cls)]=i
print(len(classes_service_type_dict))
data['QHC_FUP_SERVICE_TYPE']=data['QHC_FUP_SERVICE_TYPE'].map(lambda x: classes_service_type_dict[str(x)])
data.head(5)

# ConsumerFactor & OverallSatisfactionScore

In [ ]:
def f1(x):
    val=re.search('\[(.*?)\]',str(x))
    if val is None:
        val=None
    else:
        val=val.group(0)
        val=val.replace('[','').replace(']','').replace('/',' ')
    return val


cf_df=data.dropna()
cf_df=cf_df[['ConsumerFactor','OverallSatisfactionScore','DEFECT_DESC']]
cf_df['ConsumerFactor']=cf_df['ConsumerFactor'].map(f1)

cf_df2=pd.DataFrame(columns=['IH','CI','PS','AD'])
cf_df=cf_df.dropna()
for i in range(len(cf_df)):
    val=cf_df.iloc[i]['ConsumerFactor']
    a,b,c,d=0,0,0,0
    if 'IH' in val:
        a+=1
    if 'CI' in val:
        b+=1
    if 'PS' in val:
        c+=1
    if 'AD' in val:
        d+=1
    cf_df2 = cf_df2.append({'IH': a, 'CI': b, 'PS': c,'AD':d}, ignore_index=True)

cf_df=cf_df.dropna()
# cf_df=pd.concat([cf_df2,cf_df],axis=1)
cf_df = pd.concat([cf_df2.reset_index(drop=True),cf_df.reset_index(drop=True)],axis=1)
cf_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cf_df[['IH','CI','PS','AD']], cf_df['OverallSatisfactionScore'], test_size=0.3, random_state=42)

In [ ]:
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                     hidden_layer_sizes=(5,10,2), random_state=1)
# clf=SGDClassifier(loss="hinge", penalty="l2", max_iter=20)
# clf = RandomForestClassifier(max_depth=20, random_state=0)

kernel = 1.0 * RBF(1.0)
clf = GaussianProcessClassifier(kernel=kernel,
                                random_state=0)

res=clf.fit(X_train, y_train)
print(res)
res=clf.score(X_test,y_test)
print(res)

In [ ]:
cf_df['OverallSatisfactionScore'].hist()
plt.show()

In [ ]:
xlf = xgb.XGBRegressor(max_depth=50,
                       learning_rate=0.01,
                       objective="reg:squarederror",
                       nthread=-1,
                       gamma=0,
                       seed=1440,
                       missing=1,
                       )

In [ ]:
xlf.fit(X_train.values, y_train.values, verbose=True, eval_set=[(X_test.values, y_test.values)])

In [ ]:
y_test.values.shape

In [ ]:
preds

In [ ]:
labels=['0','1','2','3','4']
preds = xlf.predict(X_test.values)
# preds=np.array([round(x) for x in preds])

print(classification_report(y_test.values,preds,labels=labels))

# Visualization

In [ ]:
ft_path='weights/geowac_lemmas_none_fasttextskipgram_300_5_2020/model.model'
ft_model = fasttext.FastTextKeyedVectors.load(ft_path)

In [ ]:
ft_model.most_similar('экран')

In [ ]:
ft_model['экран']

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
#Mean Pooling - Take attention mask into account for correct averaging



In [ ]:
#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
#Tokenize sentences

In [ ]:
sentences = ['Привет! Как твои дела?']

# model.to('cuda')

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_emb(x):
    encoded_input = tokenizer(x, padding=True, truncation=True, max_length=24, return_tensors='pt')
    #Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    #Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    return sentence_embeddings.detach().cpu().numpy()[0]

In [ ]:
embeddings=[]
for desc in cf_df['DEFECT_DESC']:
    # x=torch.autograd.Variable(desc)
    embeddings.append(get_emb(desc))

In [ ]:
umap_2d = UMAP(random_state=86)

umaped_vct = umap_2d.fit_transform(embeddings)

In [ ]:
colors=['r','orange','blue','black','green']
clr=[colors[key] for key in cf_df['OverallSatisfactionScore']]

plt.figure(figsize=(10,10))
plt.scatter(umaped_vct[:,0],umaped_vct[:,1],c=clr)
plt.show()

In [ ]:
umaped_vct_colors=np.concatenate([umaped_vct,cf_df['OverallSatisfactionScore'].values.reshape((-1,1))],axis=1)

In [ ]:
umap_2d = UMAP(random_state=86)

umaped_vct_colors_2d = umap_2d.fit_transform(umaped_vct_colors)

In [ ]:
classes_score_dict

In [ ]:
colors=['r','orange','blue','black','green']
clr=[colors[key] for key in cf_df['OverallSatisfactionScore']]

plt.figure(figsize=(150,150))
plt.scatter(umaped_vct_colors_2d[:,0],umaped_vct_colors_2d[:,1],c=clr)
for i in range(len(umaped_vct_colors_2d[:,0])):
    plt.text(umaped_vct_colors_2d[i,0],umaped_vct_colors_2d[i,1],cf_df['DEFECT_DESC'].iloc[i])
plt.legend(['red - 0','orange - 0.166','blue - 0.167','black - 0.333','green - 0.5'],fontsize=50)
plt.savefig('map.png')
plt.show()

In [ ]:
plt.savefig('map.png')